In [10]:
import numpy as np
import pandas as pd
from sklearn import __version__ as sklearn_version

In [3]:
data = pd.read_csv("./data/rec_ab_test.csv")
data.head()

,recommender,age,tenure,watch_time
0,challenger,15,1,2.39
1,challenger,27,1,2.32
2,benchmark,17,0,2.74
3,benchmark,34,1,1.92
4,benchmark,14,1,2.47


In [4]:
import statsmodels.formula.api as smf

result = smf.ols('watch_time ~ C(recommender)', data=data).fit()

result.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,2.0491,0.058,35.367,0.000,1.935,2.163
C(recommender)[T.challenger],0.1427,0.095,1.501,0.134,-0.044,0.330


In [5]:
import dowhy
from dowhy import CausalModel
import pandas as pd
import numpy as np

In [12]:
# Convert treatment to binary
data['recommender_binary'] = (data['recommender'] == 'challenger').astype(int)

# Now create the causal model with the binary treatment
model = CausalModel(
    data=data,
    treatment='recommender_binary',  # Using our new binary variable
    outcome='watch_time',
    graph="digraph {recommender_binary -> watch_time}")

# Identify the causal effect
identified_estimand = model.identify_effect()

# Estimate the effect
estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression"
)

/Users/vaclavpechtor/Code/code_examples/causal-inference-in-python-code/env/lib/python3.11/site-packages/dowhy/causal_estimators/regression_estimator.py:179: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]


In [13]:
# Verify randomization and perform robustness checks
refute_results = model.refute_estimate(
    identified_estimand, 
    estimate,
    method_name="random_common_cause"
)

/Users/vaclavpechtor/Code/code_examples/causal-inference-in-python-code/env/lib/python3.11/site-packages/dowhy/causal_estimators/regression_estimator.py:179: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]
/Users/vaclavpechtor/Code/code_examples/causal-inference-in-python-code/env/lib/python3.11/site-packages/dowhy/causal_estimators/regression_estimator.py:179: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]
/Users/vaclavpechtor/Code/code_examples/causal-inference-in-python-code/env/lib/python3.11/site-packages/dowhy/causal_estimators/r

In [14]:
# Print the estimate details
print(estimate)

# For more detailed results
print("\nCausal Estimate:")
print("Estimated effect:", estimate.value)
print("95% CI:", estimate.get_confidence_intervals())
print("\nEffect significance:")
print(estimate.test_stat_significance())

*** Causal Estimate ***

## Identified estimand
Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
          d                        
─────────────────────(E[watchₜᵢₘₑ])
d[recommender_binary]              
Estimand assumption 1, Unconfoundedness: If U→{recommender_binary} and U→watch_time then P(watch_time|recommender_binary,,U) = P(watch_time|recommender_binary,)

## Realized estimand
b: watch_time~recommender_binary
Target units: ate

## Estimate
Mean value: 0.1426859605911326


Causal Estimate:
Estimated effect: 0.1426859605911326
95% CI: [[-0.04431842  0.32969034]]

Effect significance:
{'p_value': array([0.1343051])}
